In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torchvision import datasets
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

ChatGPT Generated Code (Given)

In [8]:
batch_size = 64
num_epochs = 5
learning_rate = 0.01
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
data=datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True) #FashionMNIST Dataset

#Splitting Data
train_size = int(0.8 * len(data))
val_size = len(data) - train_size
train_dataset, val_dataset = random_split(data, [train_size, val_size])
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

#Dataloaders
train_loader=DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader=DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader=DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

labels=("T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot")

In [9]:
class FashionMNISTModel(nn.Module):
    def __init__(self):
        super(FashionMNISTModel, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 16, 3)  # Input channels: 1 (grayscale), output channels: 16, kernel size: 3
        self.conv2 = nn.Conv2d(16, 32, 3)  # Input channels: 16, output channels: 32, kernel size: 3
        self.conv3 = nn.Conv2d(32, 64, 3)  # Input channels: 32, output channels: 64, kernel size: 3

        # Pooling layer
        self.pool = nn.MaxPool2d(2, 2)  # Kernel size: 2, stride: 2

        # Fully connected layers
        # Adjust input features based on the spatial dimensions after pooling
        self.fc1 = nn.Linear(64, 32)  # Input features: 64 * 2 * 2, output features: 256
        self.fc2 = nn.Linear(32, 10)  # Input features: 256, output features: 10 (number of Fashion MNIST classes)

        # Activation function
        self.relu = nn.ReLU()

    def forward(self, x):
        # Apply convolutional layers and pooling layers
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))

        # Flatten the output
        x = x.view(x.size(0), -1)  # Reshape to (batch_size, -1) where -1 infers the correct size

        # Apply fully connected layers
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [10]:
# Create an instance of the model
model=FashionMNISTModel()

# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    running_loss = 0.0
    running_acc = 0.0
    val_loss = 0.0
    val_acc = 0.0

    # Loop over the training batches
    for i, data in enumerate(train_loader):
        inputs, labels = data  # Get inputs and labels
        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute the loss

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Compute accuracy
        _, preds = torch.max(outputs, 1)
        acc = torch.sum(preds == labels).item() / batch_size

        # Print statistics
        running_loss += loss.item()
        running_acc += acc

        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f acc: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200, running_acc / 200))
            running_loss = 0.0
            running_acc = 0.0

    # Validation loop (similar structure as training loop)
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)
            acc = torch.sum(preds == labels).item() / batch_size

            val_loss += loss.item()
            val_acc += acc

        print('Epoch [%d] Validation loss: %.3f acc: %.3f' %
              (epoch + 1, val_loss / len(val_loader), val_acc / len(val_loader)))
        val_loss = 0.0
        val_acc = 0.0

print('Finished Training')

[1,   200] loss: 0.863 acc: 0.671
[1,   400] loss: 0.566 acc: 0.797
[1,   600] loss: 0.520 acc: 0.813
Epoch [1] Validation loss: 0.463 acc: 0.832
[2,   200] loss: 0.448 acc: 0.838
[2,   400] loss: 0.447 acc: 0.835
[2,   600] loss: 0.436 acc: 0.845
Epoch [2] Validation loss: 0.471 acc: 0.833
[3,   200] loss: 0.415 acc: 0.850
[3,   400] loss: 0.410 acc: 0.849
[3,   600] loss: 0.403 acc: 0.859
Epoch [3] Validation loss: 0.486 acc: 0.824
[4,   200] loss: 0.391 acc: 0.860
[4,   400] loss: 0.384 acc: 0.864
[4,   600] loss: 0.390 acc: 0.860
Epoch [4] Validation loss: 0.404 acc: 0.855
[5,   200] loss: 0.374 acc: 0.865
[5,   400] loss: 0.384 acc: 0.860
[5,   600] loss: 0.389 acc: 0.861
Epoch [5] Validation loss: 0.393 acc: 0.858
Finished Training


In [11]:
# Evaluate the model on the testing set
correct = 0
total = 0

# Loop over the testing batches
with torch.no_grad():
    for data in test_loader:

        # Get the inputs and the labels
        inputs, labels = data

        # Forward pass
        outputs = model(inputs)

        # Get the predicted labels by taking the argmax of the outputs
        _, predicted = torch.max(outputs.data, 1)

        # Update the total and correct counts
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate the accuracy
accuracy = 100 * correct / total

# Print the accuracy
print('Accuracy on the test set: {:.2f}%'.format(accuracy))

Accuracy on the test set: 85.15%


State of the Art Models

In [12]:
import torchvision.models as models
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [13]:
batch_size = 64
num_epochs = 5
learning_rate = 0.01
transform = transforms.Compose([transforms.Grayscale(num_output_channels=3),transforms.Resize((224, 224)),
                                transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
data = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)

#splitting dataset
train_size = int(0.8 * len(data))
val_size = len(data) - train_size
train_dataset, val_dataset = random_split(data, [train_size, val_size])
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

#Dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
labels=("T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot")

MobileNetV3

In [14]:
model=models.mobilenet_v3_small(pretrained=True)
lastLayerFtrs=model.classifier[0].in_features
model.fc=nn.Linear(lastLayerFtrs, 10)
model=model.to(device)

criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100%|██████████| 9.83M/9.83M [00:00<00:00, 49.4MB/s]


In [15]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print("Epoch:",epoch+1, "Loss:",running_loss/len(train_loader))

model.eval()
correct=0
total=0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print("Accuracy:",100 * correct / total,"%")

Epoch: 1 Loss: 0.5617301027278105
Epoch: 2 Loss: 0.318938423504432
Epoch: 3 Loss: 0.2954777308156093
Epoch: 4 Loss: 0.27251185322801275
Epoch: 5 Loss: 0.2540986368159453
Accuracy: 88.46 %


ShuffleNetV2

In [16]:
model = models.shufflenet_v2_x2_0(pretrained=True)
lastLayerFtr = model.fc.in_features
model.fc = nn.Linear(lastLayerFtr, 10)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X2_0_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X2_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/shufflenetv2_x2_0-8be3c8ee.pth" to /root/.cache/torch/hub/checkpoints/shufflenetv2_x2_0-8be3c8ee.pth
100%|██████████| 28.4M/28.4M [00:00<00:00, 77.0MB/s]


In [17]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print("Epoch:",epoch+1, "Loss:",running_loss/len(train_loader))

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print("Accuracy:",100*correct/total,"%")

Epoch: 1 Loss: 0.4992402017116547
Epoch: 2 Loss: 0.28823752696812155
Epoch: 3 Loss: 0.2507557063748439
Epoch: 4 Loss: 0.2247825993100802
Epoch: 5 Loss: 0.2052414425164461
Accuracy: 89.03 %
